In [166]:
from utils import *
import collections

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + column_units + square_units

# TODO: Update the unit list to add the new diagonal units
unitlist = unitlist


# Must be called after all units (including diagonals) are added to the unitlist
units = extract_units(unitlist, boxes)
peers = extract_peers(units, boxes)




################## custom functions ########################
def check_if_solved(values):
    ## check if sudoku is solved
    if all(len(values[key]) == 1 for key in boxes):
        return values # sudoku is solved
    else:
        False
def local_box(key,values):
    letters_cat={'A':'ABC','B':'DEF','C':'GHI'}
    numbers_cat={'1':'123','2':'456','3':'789'}
    letters_key =[dict_key for dict_key,value in letters_cat.items() if key[0] in value]
    numbers_key =[dict_key for dict_key,value in numbers_cat.items() if key[1] in value]
    return cross(letters_cat[letters_key[0]],numbers_cat[numbers_key[0]])
def remove_value_from(group_keys,numbers_to_remove,values,exception=[]):
    '''function to remove number from sudoku library'''
    ## loop over each number
    for number in numbers_to_remove:
        ## loop over the dictionary of group we are interested in
        for key in group_keys:
            # check that the field is not already completed
            if len(values[key])>1 and key not in exception:
                # remove value from the string by replacing it with ""
                values[key]=values[key].replace(number,'')
    return values
def local_row_column(key,values):
    '''function to get row of a key 
    :key: String box_name eg 'A1','A2' 
    :values: Dict Sudoku board'''
    return [box_name for box_name in values.keys() if ((key[0] in box_name) or (key[1] in box_name))]
def local_row(key,values):
    '''function to get row of a key 
    :key: String box_name eg 'A1','A2' 
    :values: Dict Sudoku board'''
    return [box_name for box_name in values.keys() if ((key[0] in box_name))]
def local_column(key,values):
    '''function to get row of a key 
    :key: String box_name eg 'A1','A2' 
    :values: Dict Sudoku board'''
    return [box_name for box_name in values.keys() if ((key[1] in box_name))]
def local_group(key,values):
    '''function to get row,column,and local box of a key 
    :key: String box_name eg 'A1','A2' 
    :values: Dict Sudoku board
    :return: list of local group key
    '''
    key_local_box= local_box(key,values)
#     print(key_local_box)
    return [box_name for box_name in values.keys() if ((key[0] in box_name) or (key[1] in box_name)) or (box_name in key_local_box)]
def elimination(values):
    new_sudoku = values.copy()
    stall=False
    while stall==False:
        original_sudoku= new_sudoku.copy()
        for key in new_sudoku.keys():
            if len(new_sudoku[key])==1:
                remove_value_from(local_group(key,new_sudoku),new_sudoku[key] ,new_sudoku ,exception=[key])
        if original_sudoku == new_sudoku:
            stall=True
    return new_sudoku
        
def check_if_in_local_group(values,key,value_to_check):
    for local_group_key in local_group(key,values):
        if (key != local_group_key) and (str(value_to_check) in values[local_group_key]):
            return True
    return False
        
def only_choice(values):
    new_sudoku = values.copy()
    stall=False
    while stall==False:
        original_sudoku= new_sudoku.copy()
        for key in new_sudoku.keys():
            #check if box not complete and list local_group keys
            if len(new_sudoku[key])>1:
                for potential_value in new_sudoku[key]:
                    if not check_if_in_local_group(values,key,potential_value):
                        new_sudoku[key]=potential_value
        if original_sudoku == new_sudoku:
            stall=True
    return new_sudoku
def naked_twins(values):
    new_sudoku = values.copy()
    stall=False
    while stall==False:
        original_sudoku= new_sudoku.copy()
        ## check rows
        for unit in unitlist:
#             print(f'unit is {unit}')
            two_value_dict = {key:new_sudoku[key] for key in unit if len(new_sudoku[key])==2}
            #check for duplicate for each key
            duplicate_values=[item for item, count in collections.Counter(two_value_dict.values()).items() if count > 1]
            if duplicate_values:
                for duplicate_value in duplicate_values:
                    copy_new_sudoku=new_sudoku.copy()
                    new_sudoku=remove_value_from(unit,duplicate_value,new_sudoku,exception=[key for key in unit if new_sudoku[key]==duplicate_value])
                    if copy_new_sudoku != new_sudoku:
                        print(f'found duplicate values of {duplicate_values}')
                        print(f'dict_before and after removing duplicats')
                        display(copy_new_sudoku)
                        display(new_sudoku)
        if original_sudoku == new_sudoku:
            stall=True
        print('loop finished')
    return new_sudoku

def reduce(values):
    new_sudoku = values.copy()
    stall=False
    while stall==False:
        original_sudoku= new_sudoku.copy()
        new_sudoku=elimination(new_sudoku)
        new_sudoku=only_choice(new_sudoku)
        new_sudoku=naked_twins(new_sudoku)
        if original_sudoku == new_sudoku:
            stall=True
    return new_sudoku
def reduce_puzzle(values):
    new_sudoku = values.copy()
    stall=False
    while stall==False:
        original_sudoku= new_sudoku.copy()
        new_sudoku=elimination(new_sudoku)
        new_sudoku=only_choice(new_sudoku)
        new_sudoku=naked_twins(new_sudoku)
        if original_sudoku == new_sudoku:
            stall=True
    return new_sudoku
def search(values):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt


In [173]:
# values = grid2values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
values = grid2values('2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3')

In [161]:
values=elimination(values)
# print(values)
# for key in values.keys():
#     for number in '123456789':
#         print(check_if_in_local_group(values,key,number))
# check_if_in_local_group(values,'C2',5)
# only_choice(values)
display(reduce(values))
# naked_twins(values)

loop finished
loop finished
 4   8   3  | 9   2   1  | 6   5   7  
269  69  7  | 3   4   5  | 8   29  1  
 29  5   1  | 8   7   6  | 4   29  3  
------------+------------+------------
 5   4   8  | 1   3   2  | 9   7   6  
 7   2   9  | 5   6   4  | 1   3   8  
 1   3   6  | 7   9   8  | 2   4   5  
------------+------------+------------
 3   7   2  | 6   8   9  | 5   1   4  
 8   1   4  | 2   5   3  | 7   6   9  
 69  69  5  | 4   1   7  | 3   8   2  



In [155]:
display(values)

 4   8   3  | 9   2   1  | 6   5   7  
269  69  7  | 3   4   5  | 8   29  1  
 29  59  1  | 8   7   6  | 4   29  3  
------------+------------+------------
 5   4   8  | 1   3   2  | 9   7   6  
 7   2   9  | 5   6   4  | 1   3   8  
 1   3   6  | 7   9   8  | 2   4   5  
------------+------------+------------
 3   7   2  | 6   8   9  | 5   1   4  
 8   1   4  | 2   5   3  | 7   6   9  
 69  69  5  | 4   1   7  | 3   8   2  



In [114]:
unitlist

[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
 ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'],
 ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
 ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'],
 ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'],
 ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'],
 ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9'],
 ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9'],
 ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9'],
 ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8'],
 ['A9', 'B9', 'C9', 'D9', 'E9',

In [40]:
keys_for_local_box=local_box('A2',values)
remove_value_from(keys_for_local_box,'1',values,['A2'])
local_group('A1',values)

['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']


['A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'B1',
 'B2',
 'B3',
 'C1',
 'C2',
 'C3',
 'D1',
 'E1',
 'F1',
 'G1',
 'H1',
 'I1']

In [7]:
values

{'A1': '123456789',
 'A2': '123456789',
 'A3': '3',
 'A4': '123456789',
 'A5': '2',
 'A6': '123456789',
 'A7': '6',
 'A8': '123456789',
 'A9': '123456789',
 'B1': '9',
 'B2': '123456789',
 'B3': '123456789',
 'B4': '3',
 'B5': '123456789',
 'B6': '5',
 'B7': '123456789',
 'B8': '123456789',
 'B9': '1',
 'C1': '123456789',
 'C2': '123456789',
 'C3': '1',
 'C4': '8',
 'C5': '123456789',
 'C6': '6',
 'C7': '4',
 'C8': '123456789',
 'C9': '123456789',
 'D1': '123456789',
 'D2': '123456789',
 'D3': '8',
 'D4': '1',
 'D5': '123456789',
 'D6': '2',
 'D7': '9',
 'D8': '123456789',
 'D9': '123456789',
 'E1': '7',
 'E2': '123456789',
 'E3': '123456789',
 'E4': '123456789',
 'E5': '123456789',
 'E6': '123456789',
 'E7': '123456789',
 'E8': '123456789',
 'E9': '8',
 'F1': '123456789',
 'F2': '123456789',
 'F3': '6',
 'F4': '7',
 'F5': '123456789',
 'F6': '8',
 'F7': '2',
 'F8': '123456789',
 'F9': '123456789',
 'G1': '123456789',
 'G2': '123456789',
 'G3': '2',
 'G4': '6',
 'G5': '123456789',
 'G6

In [15]:
extract_peers(values, ['I6','I7','I8'])

defaultdict(set,
            {'I6': {'1', '2', '3', '4', '5', '6', '7', '8', '9'},
             'I7': {'3'},
             'I8': {'1', '2', '3', '4', '5', '6', '7', '8', '9'}})

In [17]:
extract_units(values, ['I9'])

defaultdict(list, {'I9': ['I9']})

In [29]:
local_box('A2',values)

['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']

In [174]:
search(values)

loop finished
loop finished
loop finished
loop finished
found duplicate values of ['45']
dict_before and after removing duplicats
   2     356789  34789  |1345789  134578  134579 | 13569  1345689  145689 
 45789   35789   34789  |1345789  134578    6    |   2     134589  14589  
 45689   35689     1    | 34589   23458   23459  |  3569     7     45689  
------------------------+------------------------+------------------------
  4579    579      6    | 13457   123457    8    |  1359   12359    1259  
   3       1       2    |   45      9       45   |   56     568      7    
  5789    5789    789   |   6     12357   12357  |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      379   | 13579   13567   13579  |   8     12569   12569  
 16789   36789     5    |   2     134678  13479  |  1679    1469    1469  
 16789   26789    789   | 145789  145678  14579  | 15679   124569    3    

   2     356789  34789  |1345789  134578  13

{'A1': '2',
 'A2': '6',
 'A3': '4',
 'A4': '7',
 'A5': '5',
 'A6': '3',
 'A7': '1',
 'A8': '9',
 'A9': '8',
 'B1': '7',
 'B2': '3',
 'B3': '9',
 'B4': '8',
 'B5': '1',
 'B6': '6',
 'B7': '2',
 'B8': '5',
 'B9': '4',
 'C1': '8',
 'C2': '5',
 'C3': '1',
 'C4': '9',
 'C5': '4',
 'C6': '2',
 'C7': '3',
 'C8': '7',
 'C9': '6',
 'D1': '4',
 'D2': '7',
 'D3': '6',
 'D4': '1',
 'D5': '2',
 'D6': '8',
 'D7': '5',
 'D8': '3',
 'D9': '9',
 'E1': '3',
 'E2': '1',
 'E3': '2',
 'E4': '4',
 'E5': '9',
 'E6': '5',
 'E7': '6',
 'E8': '8',
 'E9': '7',
 'F1': '5',
 'F2': '9',
 'F3': '8',
 'F4': '6',
 'F5': '3',
 'F6': '7',
 'F7': '4',
 'F8': '1',
 'F9': '2',
 'G1': '6',
 'G2': '4',
 'G3': '3',
 'G4': '7',
 'G5': '6',
 'G6': '1',
 'G7': '8',
 'G8': '2',
 'G9': '5',
 'H1': '9',
 'H2': '8',
 'H3': '5',
 'H4': '2',
 'H5': '8',
 'H6': '9',
 'H7': '7',
 'H8': '4',
 'H9': '1',
 'I1': '1',
 'I2': '2',
 'I3': '7',
 'I4': '5',
 'I5': '8',
 'I6': '4',
 'I7': '9',
 'I8': '6',
 'I9': '3'}

In [191]:
values=grid2values('2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3')

In [192]:
result=search(values)

loop finished
loop finished
loop finished
loop finished
found duplicate values of ['45']
dict_before and after removing duplicats
   2     356789  34789  |1345789  134578  134579 | 13569  1345689  145689 
 45789   35789   34789  |1345789  134578    6    |   2     134589  14589  
 45689   35689     1    | 34589   23458   23459  |  3569     7     45689  
------------------------+------------------------+------------------------
  4579    579      6    | 13457   123457    8    |  1359   12359    1259  
   3       1       2    |   45      9       45   |   56     568      7    
  5789    5789    789   |   6     12357   12357  |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      379   | 13579   13567   13579  |   8     12569   12569  
 16789   36789     5    |   2     134678  13479  |  1679    1469    1469  
 16789   26789    789   | 145789  145678  14579  | 15679   124569    3    

   2     356789  34789  |1345789  134578  13

In [197]:
display(result)
print(result)
values2grid(result)

2 6 4 |7 5 3 |1 9 8 
7 3 9 |8 1 6 |2 5 4 
8 5 1 |9 4 2 |3 7 6 
------+------+------
4 7 6 |1 2 8 |5 3 9 
3 1 2 |4 9 5 |6 8 7 
5 9 8 |6 3 7 |4 1 2 
------+------+------
6 4 3 |7 6 1 |8 2 5 
9 8 5 |2 8 9 |7 4 1 
1 2 7 |5 8 4 |9 6 3 

{'A1': '2', 'A2': '6', 'A3': '4', 'A4': '7', 'A5': '5', 'A6': '3', 'A7': '1', 'A8': '9', 'A9': '8', 'B1': '7', 'B2': '3', 'B3': '9', 'B4': '8', 'B5': '1', 'B6': '6', 'B7': '2', 'B8': '5', 'B9': '4', 'C1': '8', 'C2': '5', 'C3': '1', 'C4': '9', 'C5': '4', 'C6': '2', 'C7': '3', 'C8': '7', 'C9': '6', 'D1': '4', 'D2': '7', 'D3': '6', 'D4': '1', 'D5': '2', 'D6': '8', 'D7': '5', 'D8': '3', 'D9': '9', 'E1': '3', 'E2': '1', 'E3': '2', 'E4': '4', 'E5': '9', 'E6': '5', 'E7': '6', 'E8': '8', 'E9': '7', 'F1': '5', 'F2': '9', 'F3': '8', 'F4': '6', 'F5': '3', 'F6': '7', 'F7': '4', 'F8': '1', 'F9': '2', 'G1': '6', 'G2': '4', 'G3': '3', 'G4': '7', 'G5': '6', 'G6': '1', 'G7': '8', 'G8': '2', 'G9': '5', 'H1': '9', 'H2': '8', 'H3': '5', 'H4': '2', 'H5': '8', 'H6': '9', 'H7': '7

'264753198739816254851942376476128539312495687598637412643761825985289741127584963'

In [194]:
unit_test_result=values2grid({'G7': '8', 'G6': '9', 'G5': '7', 'G4': '3', 'G3': '2', 'G2': '4', 'G1': '6', 'G9': '5',
'G8': '1', 'C9': '6', 'C8': '7', 'C3': '1', 'C2': '9', 'C1': '4', 'C7': '5', 'C6': '3',
'C5': '2', 'C4': '8', 'E5': '9', 'E4': '1', 'F1': '1', 'F2': '2', 'F3': '9', 'F4': '6',
'F5': '5', 'F6': '7', 'F7': '4', 'F8': '3', 'F9': '8', 'B4': '7', 'B5': '1', 'B6': '6',
'B7': '2', 'B1': '8', 'B2': '5', 'B3': '3', 'B8': '4', 'B9': '9', 'I9': '3', 'I8': '2',
'I1': '7', 'I3': '8', 'I2': '1', 'I5': '6', 'I4': '5', 'I7': '9', 'I6': '4', 'A1': '2',
'A3': '7', 'A2': '6', 'E9': '7', 'A4': '9', 'A7': '3', 'A6': '5', 'A9': '1', 'A8': '8',
'E7': '6', 'E6': '2', 'E1': '3', 'E3': '4', 'E2': '8', 'E8': '5', 'A5': '4', 'H8': '6',
'H9': '4', 'H2': '3', 'H3': '5', 'H1': '9', 'H6': '1', 'H7': '7', 'H4': '2', 'H5': '8',
'D8': '9', 'D9': '2', 'D6': '8', 'D7': '1', 'D4': '4', 'D5': '3', 'D2': '7', 'D3': '6',
'D1': '5'})

In [195]:
unit_test_result

'267945381853716249491823576576438192384192657129657438642379815935281764718564923'

In [196]:
unit_test_result==result

False